In [ ]:
# vertex bq gh /sdk_automl_tabular_regression_batch_bq.ipynb

In [1]:
%%capture
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    'google-cloud-bigquery[bqstorage,pandas]' \
                                    google-cloud-storage 
                                    
     

In [2]:
%%capture
!pip3 install gspread google-auth oauth2client df2gspread

In [3]:
# For connectopm to g-sheets
import gspread
# import google-auth
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

# data manipulation
import pandas as pd
import numpy as np

# general python imports
import os
from contextlib import redirect_stdout
import io
import sys
import subprocess

################################################    
# G-SHEETS CONNECTION
scope = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']

In [4]:
#authorize sa
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(credentials)
spreadsheet_key = '1CdWbcllzJWkPdUFqfIMwrwpyA7ZRiANh9ZlUws7eShM'
sh = client.open_by_key(spreadsheet_key)

In [5]:
# IMPORT VERTEX AI LIBRARIES

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery
     

In [6]:
# g-sheets setup

PROJECT_ID = sh.worksheet('Main').acell('B1').value

# print(PROJECT_ID)

REGION = sh.worksheet('Main').acell('B2').value

# print(REGION)


In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION)


In [8]:
IMPORT_FILE = sh.worksheet('Main').acell('B3').value

# print(IMPORT_FILE)

In [9]:
# %%capture
# ! gcloud auth login

In [10]:
# Prepare bnatch data predictions


# Create client in default region
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

In [11]:
# Create training dataset in default region
TRAINING_INPUT_DATASET_ID = "gsod_training_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

# Create test dataset in default region
PREDICTION_INPUT_DATASET_ID = "gsod_prediction_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")
     

Conflict: 409 POST https://bigquery.googleapis.com/bigquery/v2/projects/sad-vertex-ai/datasets?prettyPrint=false: Already Exists: Dataset sad-vertex-ai:gsod_training_unique

In [12]:
# Select top 3000 rows of dataset
TRAINING_SIZE = 3000
query = f"""
        SELECT *
        FROM {IMPORT_FILE}
        LIMIT {TRAINING_SIZE}
        """

TRAINING_INPUT_TABLE_ID = f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}.test"
job_config = bigquery.QueryJobConfig(destination=TRAINING_INPUT_TABLE_ID)

query_job = bq_client.query(query, job_config=job_config)  # API request
query_job.result()  # Waits for query to finish

In [19]:

dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data_unique",
    bq_source=[f"bq://{TRAINING_INPUT_TABLE_ID}"],
)

label_column = "mean_temp"

print(dataset.resource_name)

Creating TabularDataset
Create TabularDataset backing LRO: projects/854460851669/locations/us-central1/datasets/6857606743438917632/operations/5027306945644593152
TabularDataset created. Resource name: projects/854460851669/locations/us-central1/datasets/6857606743438917632
To use this TabularDataset in another session:
ds = aiplatform.TabularDataset('projects/854460851669/locations/us-central1/datasets/6857606743438917632')
projects/854460851669/locations/us-central1/datasets/6857606743438917632


In [20]:

COLUMN_SPECS = {
    "year": "auto",
    "month": "auto",
    "day": "auto",
}

label_column = "mean_temp"

In [21]:

training_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="job_unique",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

print(training_job)
     

In [ ]:

model = training_job.run(
    dataset=dataset,
    model_display_name="model_unique",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/291638793478340608?project=854460851669
AutoMLTabularTrainingJob projects/854460851669/locations/us-central1/trainingPipelines/291638793478340608 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/854460851669/locations/us-central1/trainingPipelines/291638793478340608 current state:
PipelineState.PIPELINE_STATE_PENDING
AutoMLTabularTrainingJob projects/854460851669/locations/us-central1/trainingPipelines/291638793478340608 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/854460851669/locations/us-central1/trainingPipelines/291638793478340608 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLTabularTrainingJob projects/854460851669/locations/us-central1/trainingPipelines/291638793478340608 current state:
PipelineState.PIPELINE_STATE_RUNNING


In [ ]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)
     

In [ ]:
# create results dataset


# Create results dataset in default region
RESULTS_DATASET_ID = "gsod_results_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{RESULTS_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

In [ ]:


dataframe = (
    bq_client.query(f"SELECT * FROM `{PREDICTION_RESULTS_DATASET_ID}.*`")
    .result()
    .to_dataframe()
)
/sdk_automl_tabular_regression_batch_bq.ipynb

print(dataframe.head())

In [ ]:

# Delete BigQuery datasets
bq_client.delete_dataset(
    f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}",
    delete_contents=True,
    not_found_ok=True,
)

bq_client.delete_dataset(
    f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}",
    delete_contents=True,
    not_found_ok=True,
)

bq_client.delete_dataset(
    f"{PROJECT_ID}.{RESULTS_DATASET_ID}", delete_contents=True, not_found_ok=True
)

# Delete Vertex AI resources
dataset.delete()
model.delete()
training_job.delete()
batch_predict_job.delete()
     